In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from IPython.display import display, clear_output

In [6]:
# Create data
np.random.seed(42)
x = np.linspace(0, 2*np.pi, 104)  

# Combine multiple sine and cosine functions
y = np.sin(x) + 0.5*np.sin(2*x) + 0.3*np.cos(3*x) + 0.6*np.sin(4*x) + 0.1*np.random.randn(104)

# Convert data to PyTorch tensors
x_tensor = torch.tensor(x, dtype=torch.float32).view(-1, 1)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# Normalize the x data for better training performance
x_min, x_max = x_tensor.min(), x_tensor.max()
x_normalized = (x_tensor - x_min) / (x_max - x_min)

In [11]:
# Define a simple neural network 
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(1, 10)  # 10 neurons in the hidden layer
        self.fc2 = nn.Linear(10, 1)

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        return self.fc2(x)

# Create the model, loss function, and an optimizer with learning rate
model = SimpleNet()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


# Creating a scatter plot of the data and a line plot for the model's predictions
trace_data = go.Scatter(x=x, y=y, mode='markers', name='Data')
trace_fit = go.Scatter(x=x, y=model(x_normalized).detach().numpy().flatten(), mode='lines', name='Fit')

num_epochs = 20000
losses = []
for epoch in range(num_epochs):
    optimizer.zero_grad()  # Reset gradients
    outputs = model(x_normalized)
    loss = criterion(outputs, y_tensor)
    loss.backward()  # Compute gradients
    optimizer.step()  # Update weights
    
    losses.append(loss.item())

    # Update the line plot
    if (epoch+1) % 100 == 0:  # Updating every 100 epochs 
        # Clear the previous figure
        clear_output(wait=True)
        
        # Update trace with the current model prediction
        trace_fit_updated = go.Scatter(
            x=x,
            y=model(x_normalized).detach().numpy().flatten(),
            mode='lines',
            name='Fit'
        )
        
        # Recreate the figure and add the traces
        fig = make_subplots(rows=1, cols=1)
        fig.add_trace(trace_data)
        fig.add_trace(trace_fit_updated)
        
        # Update layout with the new title
        fig.update_layout(title=f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}', 
                          xaxis_title='x', yaxis_title='y')
        
        # Show updated plot
        fig.show()



